In [1]:
import asyncio 
import matplotlib

import numpy as np
import pandas as pd

from matplotlib import pylab as plt
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient, resample

%matplotlib inline

In [2]:
efd_client = EfdClient('summit_efd')

Query for all the `endReadout` events on the timespan of the night.

In [3]:
t1 = Time("2020-01-25T01:11", format='isot', scale='tai')
t2 = Time("2020-01-25T01:18", format='isot', scale='tai')+TimeDelta(8.*24.*60*60., format='sec', scale='tai')

In [4]:
end_readout = await efd_client.select_time_series("lsst.sal.GenericCamera.logevent_endReadout", 
                                           ["imageName", "exposureTime"], t1, t2)

Now match each entry. For each `i` item with `intra` in the name, there must be an `i+1` with `extra` otherwise it is not a pair. 

In [5]:
intra_images = []
extra_images = []
intra_times = []
extra_times = []
intra_exptimes = []
extra_exptimes = []

i = 0
npairs = 0
nmiss = 0

while i < len(end_readout):
    intra = end_readout['imageName'][i]
    extra = end_readout['imageName'][i+1]
    
    if 'intra' in intra and 'extra' in extra:
#         print(f"Got a pair: {intra} x {extra}")
        intra_images.append(intra)
        extra_images.append(extra)
        intra_times.append(end_readout.index[i])
        extra_times.append(end_readout.index[i+1])
        intra_exptimes.append(end_readout['exposureTime'][i])
        extra_exptimes.append(end_readout['exposureTime'][i+1])
        i+=2
        npairs+=1
    else:
#         print(f"No Match: {intra} x {extra}")
        nmiss+=1
        i+=1

print(f"Got {npairs} pairs and {nmiss} misses.")

Got 36 pairs and 0 misses.


In [6]:
match_data = np.zeros(npairs, dtype=[('intra', '<U35'), 
                                     ('extra', '<U35'), 
                                     ('az', float), 
                                           ('el', float), 
                                           ('rot_pos', float),
                                           ('x', float), 
                                           ('y', float), 
                                           ('z', float), 
                                           ('u', float), 
                                           ('v', float), 
                                           ('w', float), 
                                     ('m1_pressure', float),
                                     ('dz', float)])

In [8]:
for i in range(npairs):

    
    t1 = Time(intra_times[i], scale='tai') - TimeDelta(intra_exptimes[i], format='sec', scale='tai') - TimeDelta(30., format='sec', scale='tai')
    t2 = Time(extra_times[i], scale='tai') - TimeDelta(1., format='sec', scale='tai')

    azel = await efd_client.select_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", 
                                               ["elevationCalculatedAngle99", "azimuthCalculatedAngle99"], t1, t2)
    rotator = await efd_client.select_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders",
                                                  ["nasmyth1CalculatedAngle99"], t1, t2)
    hexapod = await efd_client.select_time_series("lsst.sal.ATHexapod.command_moveToPosition",
                                                  ["x", "y", "z", "u", "v", "w"], t1, t2)
    m1_pressure = await efd_client.select_time_series("lsst.sal.ATPneumatics.m1AirPressure",
                                                  ["pressure"], t1, t2)
    offset = await efd_client.select_time_series("lsst.sal.ATAOS.command_applyAxisOffset",
                                                 ["*"], t1, t2)
    
    if len(hexapod) < 2:
        print(f"Could not get hexapod position for pair {i+1}: {intra_images[i]} x {extra_images[i]}")
        continue
    elif '1579755351' in intra_images[i]:
        print(f"{intra_times[i]}:{intra_images[i]}")
        print(hexapod)
        print(f"{extra_times[i]}:{extra_images[i]}")

    rot_pos = np.mean(rotator['nasmyth1CalculatedAngle99'])
    el = np.mean(azel['elevationCalculatedAngle99'])
    az = np.mean(azel['azimuthCalculatedAngle99'])
    x = (hexapod['x'][len(hexapod)-2]+hexapod['x'][len(hexapod)-1])/2.
    y = (hexapod['y'][len(hexapod)-2]+hexapod['y'][len(hexapod)-1])/2.
    z = (hexapod['z'][len(hexapod)-2]+hexapod['z'][len(hexapod)-1])/2.
    u = (hexapod['u'][len(hexapod)-2]+hexapod['u'][len(hexapod)-1])/2.
    v = (hexapod['v'][len(hexapod)-2]+hexapod['v'][len(hexapod)-1])/2.
    w = (hexapod['w'][len(hexapod)-2]+hexapod['w'][len(hexapod)-1])/2.
    m1 = np.mean(m1_pressure['pressure'])
    dz = round((offset['offset'][-1] - offset['offset'][-2])/2.,3)

#     print(offset['offset'][-2], offset['offset'][-1])
    match_data[i] = (f"{intra_images[i]}.fits", f"{extra_images[i]}.fits", az, el, rot_pos, x, y, z, u, u, w, m1, dz)
#     print(az, el, rot_pos, x, y, z, u, u, w)
print("done")

done


In [9]:
df = pd.DataFrame(match_data)

In [10]:
for i in range(len(df)):
    print(f"{df['intra'][i]}: {df['el'][i]+df['rot_pos'][i]}: dz={df['dz'][i]}")
# df['intra'],df['el']+df['rot_pos']

1579914693-HD32309_intra-0-1.fits: 23.90751511641109: dz=1.5
1579914974-HD32309_intra-0-1.fits: -1.317963482467377: dz=1.5
1579916016-HD32309_intra-0-1.fits: 9.835266850253532: dz=1.5
1579916251-HD32309_intra-0-1.fits: -2.4250292242683855: dz=1.5
1579916527-HD32309_intra-0-1.fits: 47.38480224676959: dz=1.5
1579917237-HD32309_intra-0-1.fits: -3.3401847378790848: dz=1.5
1579917372-HD32309_intra-0-1.fits: 43.16738412391905: dz=1.5
1579917536-HD32309_intra-0-1.fits: 90.40105396553759: dz=1.5
1579918126-HD32309_intra-0-1.fits: -0.30472694501153796: dz=1.5
1579918785-HD32309_intra-0-1.fits: 2.7512296908377607: dz=0.5
1579919548-HD32309_intra-0-1.fits: 3.94918176287095: dz=0.5
1579919938-HD32309_intra-0-1.fits: 3.825802818873683: dz=0.5
1579920024-HD32309_intra-0-1.fits: 3.740886444335686: dz=1.5
1579920116-HD32309_intra-0-1.fits: 3.633060990322093: dz=0.75
1579920264-HD32309_intra-0-1.fits: 3.4228955032293413: dz=1.0
1579921663-HD32309_intra-0-1.fits: -0.6701477201037278: dz=0.5
1579921743-H

In [11]:
df.to_csv("20200124_match_all.csv")

In [ ]:
np.savetxt("20200124_matched_data.txt", np.array(intra_images +extra_images), fmt="%s")

In [ ]:
(1.970002+0.470002)/2.

In [ ]:
(-1.029998+1.970002)/2.

In [ ]:
df2 = pd.DataFrame.from_csv("20200123_match.csv")

In [ ]:
df2 

In [ ]:
plt.plot(df2['m1_pressure'],'.')

In [ ]:
t1 = Time("2020-01-24T06:40:10.399", format='isot', scale='tai')
t2 = Time("2020-01-24T07:46:45.959", format='isot', scale='tai')

m1_pressure = await efd_client.select_time_series("lsst.sal.ATPneumatics.m1AirPressure",
                                              ["pressure"], t1, t2)



In [ ]:
fig = plt.figure(1)

ax = fig.add_subplot(111)

t1 = Time("2020-01-24T06:40:10.399", format='isot', scale='tai')
t2 = Time("2020-01-24T07:46:45.959", format='isot', scale='tai')

m1_pressure = await efd_client.select_time_series("lsst.sal.ATPneumatics.m1AirPressure",
                                              ["pressure"], t1, t2)

ax.plot(m1_pressure)

for i in [46, 47, 49, 51, 53, 59]:
    
    t1 = Time(intra_times[i], scale='tai') - TimeDelta(intra_exptimes[i], format='sec', scale='tai') - TimeDelta(30., format='sec', scale='tai')
    t2 = Time(extra_times[i], scale='tai') - TimeDelta(1., format='sec', scale='tai')

    m1_pressure = await efd_client.select_time_series("lsst.sal.ATPneumatics.m1AirPressure",
                                              ["pressure"], t1, t2)

    ax.plot(m1_pressure)
